In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import numpy as np
from datetime import datetime
import os
import glob
import shutil
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import yfinance as yf


In [3]:
# Appends new row to existing csv file
# path = "TEST/testing.csv"

def append_row_toCSV(path, toadd_data):
    mydata = pd.read_csv(path)
    newdata = pd.DataFrame([toadd_data], columns=mydata.columns)
    mydata = pd.concat([mydata, newdata], ignore_index=True)
    mydata.to_csv(path, index=False)



In [ ]:
Earnings beat/miss

Analyst ratings compared to current price
https://www.marketbeat.com/stocks/NASDAQ/AAPL/price-target/

In [151]:
# finds earnings estimate and and EPS 
def add_EPS_estimate_reported(url):
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, "html.parser")
    body = soup.find("table", class_="scroll-table sort-table", id="earnings-history")

    body = body.find("tbody").find_all("tr")
    body.pop(0)
    mylist = []
    for i in body:
        # print(i.find("td").contents)
        while True:
            date_EPS = i.find("td")
            consensus_EPS = date_EPS.find_next("td").find_next("td") 
            reported_EPS = consensus_EPS.find_next("td")
            if str(date_EPS.contents[0])[0] == "<":
                break
            date_EPS = date_EPS.contents
            consensus_EPS = consensus_EPS.contents
            reported_EPS = reported_EPS.contents
            mylist2 = [date_EPS,consensus_EPS,reported_EPS]
            mylist.append(mylist2)
            break
    return mylist

In [4]:
#check if the difference betweens two dates are below 5 to match with their respective quarterly report dates
#checks if the data on consensus/reported EPS match the quarterly report one
# since it is possible for them to be different as they could be released at different dates and depends on whether the websites I used
# to scrape from update it instantenously or not, thus ±5 days error range.
def check_day_diff(mydatestr, quarterURL):
    
    date_1 = datetime.strptime(mydatestr, f'%m/%d/%Y')

    myurl = pd.read_csv(quarterURL)
    date_string = myurl.loc[1][0]
    date_2 = datetime.strptime(date_string, f'%Y-%m-%d %H:%M:%S')

    day_difference = abs((date_2 - date_1).days)
    if day_difference < 5:
        return True
    else:
        return False

In [264]:
# sorts directory into a list, then add consensus and reported EPS to each stock's csv files
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        all_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
        all_files.sort(reverse=True) #gives a SORTED list of all .csv paths of that particular stock 
        url = 'https://www.marketbeat.com/stocks/NASDAQ/'+"MSFT"+'/earnings/'
        eps_list = add_EPS_estimate_reported(url)
        print(eps_list)
        i=0
        try:
            for quarterURL in all_files: #loops around each .csv path
                date = ''.join(eps_list[i][0])
                mybool = check_day_diff(date, quarterURL) #checks if the data on consensus/reported EPS match the quarterly report one
                consensus_EPS = ''.join(eps_list[i][1]).replace("$","") #gets consensus data, turning it into a string in the format i.e. $0.48 then getting rid of the $ sign --> to get 0.48
                reported_EPS = ''.join(eps_list[i][2]).replace("$","")
                
                percentage = (float(reported_EPS)/float(consensus_EPS))-1
                print(percentage)
                
                if mybool == True:
                    path = "TEST/testing.csv"
                    #path = quarterURL
                    append_row_toCSV(path, percentage) #adds percentage of surprise to csv file
                i+=1
        except:
            pass
        break
    break


[[['10/25/2022'], ['$2.29'], ['$2.35']], [['7/26/2022'], ['$2.28'], ['$2.23']], [['4/26/2022'], ['$2.18'], ['$2.22']], [['1/25/2022'], ['$2.29'], ['$2.48']], [['10/25/2021'], ['$2.08'], ['$2.27']], [['7/26/2021'], ['$1.92'], ['$2.17']], [['4/27/2021'], ['$1.76'], ['$1.95']], [['1/25/2021'], ['$1.64'], ['$2.03']], [['10/27/2020'], ['$1.53'], ['$1.82']], [['7/22/2020'], ['$1.34'], ['$1.46']], [['4/29/2020'], ['$1.27'], ['$1.40']], [['1/29/2020'], ['$1.32'], ['$1.51']], [['10/23/2019'], ['$1.24'], ['$1.38']], [['7/18/2019'], ['$1.21'], ['$1.37']], [['4/24/2019'], ['$1.00'], ['$1.1375']], [['1/30/2019'], ['$1.09'], ['$1.10']], [['10/24/2018'], ['$0.96'], ['$1.14']], [['7/19/2018'], ['$1.08'], ['$1.13']], [['4/26/2018'], ['$0.85'], ['$0.95']], [['1/31/2018'], ['$0.86'], ['$0.96']], [['10/26/2017'], ['$0.72'], ['$0.84']], [['7/20/2017'], ['$0.71'], ['$0.98']], [['4/27/2017'], ['$0.69'], ['$0.73']], [['1/26/2017'], ['$0.79'], ['$0.83']], [['10/20/2016'], ['$0.68'], ['$0.76']], [['7/19/2016'],

Used yfinance instead of scraping it myself since the combination of url of each stock on marketbeat is different
i.e. 
NASDAQ/MSFT/earnings
NYSE/JPM/earnings

also yfinance has more historical data (see below compared to above), both MSFT


In [2]:
import yfinance as yf


msft = yf.Ticker("msft")
earnings_list = msft.get_earnings_dates(limit=200)
earnings_list.reset_index(inplace=True)
earnings_list = earnings_list[earnings_list['Surprise(%)'].notna()]
eps_list = earnings_list.reset_index(drop=True)

eps_list[0:50]

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%)
0,2022-10-25 12:00:00-04:00,2.30,2.35,0.0204
1,2022-07-26 12:00:00-04:00,2.29,2.23,-0.0275
2,2022-04-26 12:00:00-04:00,2.19,2.22,0.0160
3,2022-01-25 11:00:00-05:00,2.31,2.48,0.0727
4,2021-10-26 12:00:00-04:00,2.08,2.27,0.0940
5,2021-07-27 12:00:00-04:00,1.92,2.17,0.1326
6,2021-04-27 12:00:00-04:00,1.78,2.03,0.1411
7,2021-01-26 11:00:00-05:00,1.64,2.03,0.2363
8,2020-10-27 12:00:00-04:00,1.54,1.82,0.1788
9,2020-07-22 12:00:00-04:00,1.34,1.46,0.0871


In [2]:
# case 1:
# csv (date_csv)      yfinance (date_yfinance)
# 2020              2022
# 2020              2021 
# 2020              2020 (MATCH)
# 
# case 2: 
# csv (date_csv)      yfinance (date_yfinance)
# 2022              2020 
# 2021              2020 
# 2020              2020 (MATCH)

#Adjusted for Yfinance
#check if the difference betweens two dates are below 5 to match with their respective quarterly report dates
def check_day_diff(date_yfinance,QuarterURL):
    myurl = pd.read_csv(QuarterURL)
    date_string = myurl.loc[1][0]
    date_csv = datetime.strptime(date_string, f'%Y-%m-%d %H:%M:%S')
    day_difference = (date_csv - date_yfinance).days

    if abs(day_difference) < 5: # if the absolute value of day difference between the csv date and yfinance date is below 5, then the quarterly csv file matches the yfinance data
        return "match"

    elif day_difference > 0: # if the date difference between the csv date and yfinance date is larger than 0, then follow case 2
        return "case 2"

    elif day_difference < 0: # if the date difference between the csv date and yfinance date is below than 0, then follow case 1
        return "case 1"


In [3]:
# sorts directory into a list, then add the difference between consensus and reported EPS to each stock's csv files
def get_EPS(stock, sector):
    all_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
    all_files.sort(reverse=True) #gives a SORTED list of all .csv paths of that particular stock 
    try: # first exception handling checks if the yfinance API is able to retrieve the data for this particular stock, if not see exception handling below
        yf_stock = yf.Ticker(stock)
        earnings_list = yf_stock.get_earnings_dates(limit=200) #uses yfinance and gets historical eps consensus/reported data
        earnings_list.reset_index(inplace=True)
        earnings_list = earnings_list[earnings_list['Surprise(%)'].notna()]
        eps_list = earnings_list.reset_index(drop=True)
    except: # if exception is thrown, yfinance couldn't find the data, it might be because of a temporary API error so the stock ticker symbol is stored in the file not_found_stock.txt, so it could be ran again later
        print("ERROR", stock, "NOT FOUND!!!!!!") 
        with open("not_found_list_stocks.txt", "a") as file_object: #writes to file a list of stocks' EPS that couldn't be found on yfinance
            file_object.write(sector)
            file_object.write("\n")
            file_object.write(stock)
            file_object.write("\n")
        return


    index_csv = 0
    index_yfinance= 0
    error_count = 0
    try: # second exception handling, see below for explanation
        while index_csv < len(all_files): # while the current csv index is below the number of all csvs --> then it loops around each csv path
            # gets the path of the csv file
            path = all_files[index_csv]
            
            # finds the EPS % surprise by referencing the index_yfinance variable
            percentage_surprise = float(eps_list['Surprise(%)'][index_yfinance])
            # finds which date the EPS data is added on yfinance
            date_yfinance = eps_list['Earnings Date'][index_yfinance]

            # fixes the "Cannot subtract tz-naive and tz-aware datetime-like objects" error
            date_yfinance = date_yfinance.replace(tzinfo=None)

            # this function checks if the date on provided by yfinance match the date on the csv file
            mycase = check_day_diff(date_yfinance, path)

            #reads the csv file and returns as a dataframe to be used for later
            read_csv = pd.read_csv(path)

            # if in the function check_day_diff returns match, in other words, if the date on yfinance matches the date on the csv file.
            if mycase == "match":
                # furthermore if the csv file dataframe has more than 4 rows (more than 3 index), the dataframe will delete every value beyond the 4th row
                # this is because:
                # index 0 is reserved for the year,
                # index 1 is reserved for the quarter,
                # index 2 is reserved for the date in which the earnings transcript is reported
                # index 3 is reserved for the content of the earnings transcript
                # index 4 is reserved for the EPS surprise value (the value we are trying to get)
                # there is a chance that index 4 returns NaN (see explanation for 1st exception handling)
                # so when the function is "re-ran" with reference to the not_found_stock.txt file, the new value of the EPS data does not "fill" the spaces for index 5 and beyond
                # so it's safe to wipe everything beyond the 3rd index, and then add the EPS value 
                
                if len(read_csv) > 3:
                    read_csv = read_csv[:3]
                index_csv += 1
                index_yfinance += 1
                
                # EPS surprise is printed on index 4
                newdata = pd.DataFrame([percentage_surprise], columns=read_csv.columns)
                read_csv = pd.concat([read_csv, newdata], ignore_index=True)
                read_csv.to_csv(path, index=False)

            elif mycase == "case 2": 
                # in the event of "case 2", the date on the csv is bigger than the date on yfinance
                # so we need the csv date to decrease while the eps date stays the same, until they match
                # to decrease date_csv, the "index_csv" will have to increment since the list is in a descending order
                index_csv += 1
                if len(read_csv) > 3:
                    read_csv = read_csv[:3]
                newdata = pd.DataFrame([np.NaN], columns=read_csv.columns)
                read_csv = pd.concat([read_csv, newdata], ignore_index=True)
                read_csv.to_csv(path, index=False)
                
            elif mycase == "case 1":
                index_yfinance += 1

    # the second exception is thrown when the below condition's index is out of bounds
    # percentage_surprise = float(eps_list['Surprise(%)'][index_yfinance])
    # the reason that it's out of bound is because the length of data yfinance provides is smaller than the csv file
    # for example for the stock XPEL, yfinance API returns the following:
    #                    Earnings Date  EPS Estimate  Reported EPS  Surprise(%)
    # 0  2022-11-09 03:00:00-05:00          0.44          0.48       0.0909
    # 1  2022-08-09 04:00:00-04:00          0.34          0.43       0.2760
    # 2  2022-05-10 04:00:00-04:00          0.24          0.28       0.1814
    # 3  2022-02-28 02:00:00-05:00          0.30          0.22      -0.2763
    # 4  2021-11-09 03:00:00-05:00          0.32          0.32       0.0127
    # 5  2021-08-09 04:00:00-04:00          0.26          0.37       0.4453
    # 6  2021-05-10 03:00:00-04:00          0.18          0.25       0.4286
    # 7  2021-03-11 02:00:00-05:00          0.20          0.22       0.1282
    # 8  2020-11-10 12:00:00-05:00          0.17          0.24       0.4118
    # 9  2020-08-12 03:00:00-04:00          0.05          0.14       1.8000
    # 10 2020-05-14 04:00:00-04:00          0.06          0.06       0.0909
    # 11 2020-03-16 04:00:00-04:00          0.10          0.17       0.7000
    # 12 2019-11-08 11:00:00-05:00          0.11          0.16       0.4545
    #
    #
    # however, the csv file (transcript data) contains data between 2022_Q3 to 2015_Q1
    # in this case, the exception is thrown.
    # When the exception is thrown, the rest of the csv files will append the value "NaN" to make sure index 4 of the csv file is filled with a value.
    except: 
        rest_of_files = all_files[index_csv:]
        for path in rest_of_files:
            read_csv = pd.read_csv(path)
            if len(read_csv) > 3:
                    read_csv = read_csv[:3]
            newdata = pd.DataFrame([np.NaN], columns=read_csv.columns)
            read_csv = pd.concat([read_csv, newdata], ignore_index=True)
            read_csv.to_csv(path, index=False)
        
# Case 1 and Case 2:

# case 1:
# csv (date_csv)      yfinance (date_yfinance)
# 2020              2022
# 2020              2021 
# 2020              2020 --> Match!
# 
# case 2: 
# csv (date_csv)      yfinance (date_yfinance)
# 2022              2020 
# 2021              2020 
# 2020              2020 --> Match!
#
# ------------------------------------------------------------------------------------------------
#
# some trace tables below with different scenarios:
#
# in the case that there's a missing value in csv_list such:
# csv_list = [2021_Q1, 2020_Q4, 2020_Q2, 2020_Q1, 2019_Q4]
# yfinance_list = [2021_Q1, 2020_Q4, 2020_Q3, 2020_Q2, 2020_Q1, 2019_Q4]
#
# we see that 2020_Q3 doesn't appear in csv_list:
#
# csv (date_csv)      yfinance (date_yfinance)
# 2021_Q1             2021_Q1  --> Match!
# 2020_Q4             2020_Q4  --> Match!
# 2020_Q2             2020_Q3  --> case 1, so date_yfinance decreases, np.NaN does not append anywhere
# 2020_Q2             2020_Q2  --> Match!
#
# ------------------------------------------------------------------------------------------------
#
# in the case that there's a missing value in yfinance_list such:
# csv_list = [2021_Q1, 2020_Q4, 2020_Q3, 2020_Q2, 2020_Q1, 2019_Q4]
# yfinance_list = [2021_Q1, 2020_Q4, 2020_Q2, 2020_Q1, 2019_Q4]

# csv (date_csv)      yfinance (date_yfinance)
# 2021_Q1             2021_Q1  --> Match!
# 2020_Q4             2020_Q4  --> Match!
# 2020_Q3             2020_Q2  --> case 2, np.NaN is appended to 2020_Q3.csv 
# 2020_Q2             2020_Q2  --> Match! 
# 2020_Q1             2020_Q1  --> Match!
# 2019_Q4             2019_Q4  --> Match!
#
# ------------------------------------------------------------------------------------------------
#
# in the case that there's a missing value in yfinance_list such:
# csv_list = [2021_Q1, 2020_Q4, 2020_Q3, 2020_Q2, 2020_Q1, 2019_Q4]
# yfinance_list = [2019_Q4, 2019_Q3, 2019_Q2, 2019_Q1]

# 2021_Q1             2019_Q4  --> case 2, np.NaN is appended to 2021_Q1.csv 
# 2020_Q4             2019_Q4  --> case 2, np.NaN is appended to 2020_Q4.csv 
# 2020_Q3             2019_Q4  --> case 2, np.NaN is appended to 2020_Q3.csv 
# 2020_Q2             2019_Q4  --> case 2, np.NaN is appended to 2020_Q2.csv  
# 2020_Q1             2019_Q4  --> case 2, np.NaN is appended to 2020_Q1.csv 
# 2019_Q4             2019_Q4  --> Match!

In [ ]:
#Adjusted for Yfinance
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    print(sector, "sector start ................................................................")
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        print(stock, "start")
        get_EPS(stock, sector) #calls get_EPS function and notes EPS info on the relevant .csv files
        print(stock, "end")
    break

In [ ]:
# I noticed that some EPS info of certain stocks were termporarily unavailable on the yfinance API.
# So I gathered a list of all stocks with the error "CAT: No earnings dates found, symbol may be delisted" onto a text file labelled not_found_list_stocks.txt
# The EPS data is then re-gathered by looping over each ticker of the not_found_list_stocks.txt file

In [3]:
# this modified get EPS function does not write to not_found_list_stocks.txt when the first exception is thrown

def modified_get_EPS(stock, sector):
    all_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
    all_files.sort(reverse=True) 
    try: 
        yf_stock = yf.Ticker(stock)
        earnings_list = yf_stock.get_earnings_dates(limit=200) #uses yfinance and gets historical eps consensus/reported data
        earnings_list.reset_index(inplace=True)
        earnings_list = earnings_list[earnings_list['Surprise(%)'].notna()]
        eps_list = earnings_list.reset_index(drop=True)
    except: 
        print("ERROR", stock, "NOT FOUND!!!!!!") 
        # with open("not_found_list_stocks.txt", "a") as file_object: #writes to file a list of stocks' EPS that couldn't be found on yfinance
        #     file_object.write(sector)
        #     file_object.write("\n")
        #     file_object.write(stock)
        #     file_object.write("\n")
        return False
    index_csv = 0
    index_yfinance= 0
    error_count = 0
    try: 
        while index_csv < len(all_files):
            path = all_files[index_csv]
            percentage_surprise = float(eps_list['Surprise(%)'][index_yfinance])
            date_yfinance = eps_list['Earnings Date'][index_yfinance]
            date_yfinance = date_yfinance.replace(tzinfo=None)
            mycase = check_day_diff(date_yfinance, path)
            read_csv = pd.read_csv(path)

            if mycase == "match":
                if len(read_csv) > 3:
                    read_csv = read_csv[:3]
                index_csv += 1
                index_yfinance += 1                
                newdata = pd.DataFrame([percentage_surprise], columns=read_csv.columns)
                read_csv = pd.concat([read_csv, newdata], ignore_index=True)
                read_csv.to_csv(path, index=False)

            elif mycase == "case 2": 
                index_csv += 1
                if len(read_csv) > 3:
                    read_csv = read_csv[:3]
                newdata = pd.DataFrame([np.NaN], columns=read_csv.columns)
                read_csv = pd.concat([read_csv, newdata], ignore_index=True)
                read_csv.to_csv(path, index=False)
                
            elif mycase == "case 1":
                index_yfinance += 1
    except: 
        rest_of_files = all_files[index_csv:]
        for path in rest_of_files:
            read_csv = pd.read_csv(path)
            if len(read_csv) > 3:
                    read_csv = read_csv[:3]
            newdata = pd.DataFrame([np.NaN], columns=read_csv.columns)
            read_csv = pd.concat([read_csv, newdata], ignore_index=True)
            read_csv.to_csv(path, index=False)
    
    return True
        

In [4]:
with open("not_found_list_stocks.txt", "r") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    mylist = f.read().splitlines()

print(mylist)

for i in range(0, len(mylist), 2):
    sector = mylist[i]
    stock = mylist[i+1]
    boolean = modified_get_EPS(stock, sector)
    print(stock, boolean)
    if boolean == True:
        mylist[i] = ""
        mylist[i+1] = ""

mylist = list(filter(None, mylist)) # deletes all empty strings (i.e. stocks that have been replaced from the text "not found list text file"

mystr = '\n'.join(mylist)

with open("not_found_list_stocks.txt", "w") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    f.write(mystr)

['automobiles', 'TEN', 'automobiles', 'XL', 'banks', 'ABTX', 'banks', 'CASH', 'banks', 'OSBC', 'banks', 'BAC', 'banks', 'BOKF', 'banks', 'SIVB', 'banks', 'BANC', 'banks', 'NBTB', 'banks', 'FBC', 'banks', 'MTG', 'banks', 'HBAN', 'banks', 'WSBC', 'banks', 'CNOB', 'banks', 'TBK', 'banks', 'UVSP', 'banks', 'WD', 'banks', 'PPBI', 'banks', 'RKT', 'banks', 'ACT', 'banks', 'KEY', 'banks', 'CFR', 'banks', 'PFSI', 'banks', 'QCRH', 'banks', 'CADE', 'banks', 'COOP', 'banks', 'VLY', 'banks', 'ONB', 'banks', 'SI', 'banks', 'SFBS', 'banks', 'UCBI', 'banks', 'AUB', 'banks', 'SBNY', 'banks', 'WBS', 'banks', 'SFNC', 'banks', 'FRME', 'banks', 'UMBF', 'banks', 'ZION', 'banks', 'CATY', 'banks', 'SNV', 'banks', 'IBTX', 'banks', 'BOH', 'banks', 'NMIH', 'banks', 'OBNK', 'banks', 'WSFS', 'banks', 'AMAL', 'banks', 'HOPE', 'banks', 'TRST', 'banks', 'BHLB', 'banks', 'JPM', 'banks', 'PNC', 'banks', 'TFSL', 'banks', 'RDN', 'banks', 'HAFC', 'capital-goods', 'CAT', 'capital-goods', 'ARRY', 'capital-goods', 'CSWI', 'c

In [5]:
with open("not_found_list_stocks.txt", "r") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    mylist = f.read().splitlines()

print(mylist)

for i in range(0, len(mylist), 2):
    sector = mylist[i]
    stock = mylist[i+1]
    boolean = modified_get_EPS(stock, sector)
    print(stock, boolean)
    if boolean == True:
        mylist[i] = ""
        mylist[i+1] = ""

mylist = list(filter(None, mylist)) # deletes all empty strings (i.e. stocks that have been replaced from the text "not found list text file"

mystr = '\n'.join(mylist)

with open("not_found_list_stocks.txt", "w") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    f.write(mystr)

['automobiles', 'TEN', 'automobiles', 'XL', 'banks', 'ABTX', 'banks', 'FBC', 'banks', 'TBK', 'capital-goods', 'FBHS', 'commercial-services', 'MANT', 'commercial-services', 'NLSN', 'consumer-durables', 'VOXX', 'consumer-services', 'HLG', 'consumer-services', 'VAC', 'consumer-services', 'TMX', 'consumer-services', 'CAKE', 'consumer-services', 'RGS', 'consumer-services', 'SEAS', 'consumer-services', 'EXPE', 'consumer-services', 'TNL', 'consumer-services', 'STON', 'diversified-financials', 'BXMT', 'diversified-financials', 'PRG', 'diversified-financials', 'CG', 'diversified-financials', 'LC', 'diversified-financials', 'EQH', 'diversified-financials', 'GCMG', 'diversified-financials', 'FINV', 'diversified-financials', 'LU', 'diversified-financials', 'NNI', 'diversified-financials', 'CBOE', 'diversified-financials', 'GDOT', 'diversified-financials', 'ENVA', 'diversified-financials', 'SNEX', 'diversified-financials', 'FHI', 'diversified-financials', 'LAZ', 'diversified-financials', 'NOAH', 'd

In [6]:
with open("not_found_list_stocks.txt", "r") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    mylist = f.read().splitlines()

print(mylist)

for i in range(0, len(mylist), 2):
    sector = mylist[i]
    stock = mylist[i+1]
    boolean = modified_get_EPS(stock, sector)
    print(stock, boolean)
    if boolean == True:
        mylist[i] = ""
        mylist[i+1] = ""

mylist = list(filter(None, mylist)) # deletes all empty strings (i.e. stocks that have been replaced from the text "not found list text file"

mystr = '\n'.join(mylist)

with open("not_found_list_stocks.txt", "w") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    f.write(mystr)

['automobiles', 'TEN', 'automobiles', 'XL', 'banks', 'ABTX', 'banks', 'FBC', 'banks', 'TBK', 'capital-goods', 'FBHS', 'commercial-services', 'MANT', 'commercial-services', 'NLSN', 'consumer-services', 'HLG', 'consumer-services', 'TMX', 'consumer-services', 'STON', 'energy', 'CLR', 'food-beverage-tobacco', 'LNDC', 'healthcare', 'HNGR', 'healthcare', 'TMCI', 'healthcare', 'THC', 'healthcare', 'AGTI', 'healthcare', 'STE', 'healthcare', 'PEN', 'healthcare', 'RDNT', 'healthcare', 'SILK', 'healthcare', 'ATRC', 'healthcare', 'CHNG', 'healthcare', 'CERT', 'healthcare', 'NXGN', 'healthcare', 'CERS', 'healthcare', 'ACHC', 'healthcare', 'CRVL', 'healthcare', 'LHCG', 'healthcare', 'MD', 'healthcare', 'CVET', 'insurance', 'SIGI', 'insurance', 'KINS', 'insurance', 'AON', 'insurance', 'AFG', 'insurance', 'THG', 'materials', 'GCP', 'materials', 'HCC', 'materials', 'AA', 'materials', 'GPK', 'materials', 'GORO', 'materials', 'ESI', 'materials', 'TMST', 'materials', 'ASH', 'materials', 'RGLD', 'materials

In [7]:
with open("not_found_list_stocks.txt", "r") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    mylist = f.read().splitlines()

print(mylist)

for i in range(0, len(mylist), 2):
    sector = mylist[i]
    stock = mylist[i+1]
    boolean = modified_get_EPS(stock, sector)
    print(stock, boolean)
    if boolean == True:
        mylist[i] = ""
        mylist[i+1] = ""

mylist = list(filter(None, mylist)) # deletes all empty strings (i.e. stocks that have been replaced from the text "not found list text file"

mystr = '\n'.join(mylist)

with open("not_found_list_stocks.txt", "w") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    f.write(mystr)

['automobiles', 'TEN', 'automobiles', 'XL', 'banks', 'ABTX', 'banks', 'FBC', 'banks', 'TBK', 'capital-goods', 'FBHS', 'commercial-services', 'MANT', 'commercial-services', 'NLSN', 'consumer-services', 'HLG', 'consumer-services', 'TMX', 'consumer-services', 'STON', 'energy', 'CLR', 'food-beverage-tobacco', 'LNDC', 'healthcare', 'HNGR', 'healthcare', 'CHNG', 'healthcare', 'CVET', 'materials', 'GCP', 'pharmaceuticals-biotech', 'GBT', 'pharmaceuticals-biotech', 'CCXI', 'pharmaceuticals-biotech', 'MTD', 'pharmaceuticals-biotech', 'AGIO', 'pharmaceuticals-biotech', 'PCRX', 'pharmaceuticals-biotech', 'ISEE', 'real-estate', 'BRG', 'real-estate', 'CTT', 'real-estate', 'APTS', 'real-estate', 'EQC', 'retail', 'JOAN', 'retail', 'WEYS', 'retail', 'MYTE', 'retail', 'PDD', 'semiconductors', 'KOPN', 'semiconductors', 'CYBE', 'semiconductors', 'KLIC', 'semiconductors', 'SQNS', 'semiconductors', 'PLAB', 'semiconductors', 'CEVA', 'semiconductors', 'SKYT', 'semiconductors', 'TSEM', 'semiconductors', 'SWKS

In [8]:
with open("not_found_list_stocks.txt", "r") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    mylist = f.read().splitlines()

print(mylist)

for i in range(0, len(mylist), 2):
    sector = mylist[i]
    stock = mylist[i+1]
    boolean = modified_get_EPS(stock, sector)
    print(stock, boolean)
    if boolean == True:
        mylist[i] = ""
        mylist[i+1] = ""

mylist = list(filter(None, mylist)) # deletes all empty strings (i.e. stocks that have been replaced from the text "not found list text file"

mystr = '\n'.join(mylist)

with open("not_found_list_stocks.txt", "w") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    f.write(mystr)

['automobiles', 'TEN', 'automobiles', 'XL', 'banks', 'ABTX', 'banks', 'FBC', 'banks', 'TBK', 'capital-goods', 'FBHS', 'commercial-services', 'MANT', 'commercial-services', 'NLSN', 'consumer-services', 'HLG', 'consumer-services', 'TMX', 'consumer-services', 'STON', 'energy', 'CLR', 'food-beverage-tobacco', 'LNDC', 'healthcare', 'HNGR', 'healthcare', 'CHNG', 'healthcare', 'CVET', 'materials', 'GCP', 'pharmaceuticals-biotech', 'GBT', 'pharmaceuticals-biotech', 'CCXI', 'real-estate', 'BRG', 'real-estate', 'CTT', 'real-estate', 'APTS', 'retail', 'JOAN', 'semiconductors', 'CYBE', 'software', 'PING', 'software', 'AVLR', 'software', 'SWCH', 'software', 'ZEN', 'software', 'CTXS', 'tech', 'MOVE', 'transportation', 'GLG']
- TEN: No summary info found, symbol may be delisted
ERROR TEN NOT FOUND!!!!!!
TEN False
- XL: No summary info found, symbol may be delisted
ERROR XL NOT FOUND!!!!!!
XL False
- ABTX: No summary info found, symbol may be delisted
ERROR ABTX NOT FOUND!!!!!!
ABTX False
- FBC: No su

In [10]:
with open("not_found_list_stocks.txt", "r") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    mylist = f.read().splitlines()

print(mylist)

for i in range(0, len(mylist), 2):
    sector = mylist[i]
    stock = mylist[i+1]
    boolean = modified_get_EPS(stock, sector)
    print(stock, boolean)
    if boolean == True:
        mylist[i] = ""
        mylist[i+1] = ""

mylist = list(filter(None, mylist)) # deletes all empty strings (i.e. stocks that have been replaced from the text "not found list text file"

mystr = '\n'.join(mylist)

with open("not_found_list_stocks.txt", "w") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    f.write(mystr)

['automobiles', 'TEN', 'automobiles', 'XL', 'banks', 'ABTX', 'banks', 'FBC', 'banks', 'TBK', 'capital-goods', 'FBHS', 'commercial-services', 'MANT', 'commercial-services', 'NLSN', 'consumer-services', 'HLG', 'consumer-services', 'TMX', 'consumer-services', 'STON', 'food-beverage-tobacco', 'LNDC', 'healthcare', 'HNGR', 'healthcare', 'CHNG', 'healthcare', 'CVET', 'materials', 'GCP', 'pharmaceuticals-biotech', 'GBT', 'pharmaceuticals-biotech', 'CCXI', 'real-estate', 'BRG', 'real-estate', 'CTT', 'real-estate', 'APTS', 'semiconductors', 'CYBE', 'software', 'PING', 'software', 'AVLR', 'software', 'SWCH', 'software', 'ZEN', 'software', 'CTXS', 'tech', 'MOVE', 'transportation', 'GLG']
- TEN: No summary info found, symbol may be delisted
ERROR TEN NOT FOUND!!!!!!
TEN False
- XL: No summary info found, symbol may be delisted
ERROR XL NOT FOUND!!!!!!
XL False
- ABTX: No summary info found, symbol may be delisted
ERROR ABTX NOT FOUND!!!!!!
ABTX False
- FBC: No summary info found, symbol may be del